In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_40.csv')
pred2 = dd.read_csv('test_pred_41.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 33.7 s, sys: 3.44 s, total: 37.1 s
Wall time: 7.77 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 163 ms, sys: 291 ms, total: 454 ms
Wall time: 453 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 276 µs, sys: 36 µs, total: 312 µs
Wall time: 317 µs


In [7]:
coef_lgb = 0.75
coef_mlp = 0.25

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 286 ms, sys: 558 ms, total: 845 ms
Wall time: 843 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000162,0.363145,0.000329,0.198633,0.112030,0.000375,0.116565,0.000253,0.001393,0.015290,0.002222,0.188844,0.000242,0.000518,0.246076
3492886,130787971,0.000176,0.013725,0.000162,0.055945,0.175157,0.000211,0.022324,0.344901,0.000343,0.036548,0.000757,0.343566,0.000259,0.005926,0.232179
3492887,130787974,0.000720,0.022006,0.000130,0.842032,0.030621,0.000314,0.090330,0.000044,0.000134,0.001318,0.000340,0.009953,0.000178,0.001879,0.104914
3492888,130788053,0.000056,0.732645,0.000071,0.145626,0.003477,0.000106,0.010364,0.076132,0.000421,0.000383,0.000248,0.029685,0.000099,0.000686,0.158265
3492889,130788054,0.000184,0.081282,0.000271,0.405340,0.191037,0.000447,0.055281,0.000107,0.000482,0.020992,0.001068,0.242159,0.000595,0.000756,0.236731


In [13]:
%%time
test_ensemble.to_csv('test_pred_43.csv',index=False)

CPU times: user 1min 28s, sys: 1.02 s, total: 1min 29s
Wall time: 1min 29s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_43.csv -m "Message"

100%|██████████████████████████████████████| 1.06G/1.06G [04:25<00:00, 4.30MB/s]
Successfully submitted to PLAsTiCC Astronomical Classification